In [31]:
import pandas as pd
d = {"headers":{"Failed":["fail_date"],"Succeeded":["success_date"]},"rows":{"Failed":[["2018-12-28"],["2018-12-29"],["2019-01-04"],["2019-01-05"]],"Succeeded":[["2018-12-30"],["2018-12-31"],["2019-01-01"],["2019-01-02"],["2019-01-03"],["2019-01-06"]]}}
pd.DataFrame(d['rows']['Failed'], columns=d['headers']['Failed']).to_csv("./Failed.txt", index=None)
pd.DataFrame(d['rows']['Succeeded'], columns=d['headers']['Succeeded']).to_csv("./Succeeded.txt", index=None)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("report contiguous dates").config("pyspark.sql.shuffle.partition", "4").getOrCreate()

In [32]:
f = spark.read.option("header", True).csv("./Failed.txt")
s = spark.read.option("header", True).csv("./Succeeded.txt")
f = f.withColumn("fail_date", f.fail_date.cast("date"))
s = s.withColumn("success_date", s.success_date.cast("date"))
f.show()
s.show()

+----------+
| fail_date|
+----------+
|2018-12-28|
|2018-12-29|
|2019-01-04|
|2019-01-05|
+----------+

+------------+
|success_date|
+------------+
|  2018-12-30|
|  2018-12-31|
|  2019-01-01|
|  2019-01-02|
|  2019-01-03|
|  2019-01-06|
+------------+



A system is running one task every day. Every task is independent of the previous tasks. The tasks can fail or succeed.

Write an SQL query to generate a report of period_state for each continuous interval of days in the period from 2019-01-01 to 2019-12-31.

period_state is 'failed' if tasks in this interval failed or 'succeeded' if tasks in this interval succeeded. Interval of days are retrieved as start_date and end_date.

Return the result table ordered by start_date.

The query result format is in the following example.



In [38]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, date_add, lit, to_date, datediff, col,date_format
import pyspark.sql.functions as F

window_spec = Window.orderBy('fail_date')
window_spec_s = Window.orderBy('success_date')
# f.select("*", to_date(lit('2019-01-01')).alias('first_date'),row_number().over(window_spec).alias("rn")).select(col("fail_date"), date_add("first_date", 1)).show()

grouped = f.where(date_format("fail_date","y")==2019).select("*", date_format("fail_date", "D").alias('day_of_year'), row_number().over(window_spec).alias("rn")).select("fail_date", (col('day_of_year')-col('rn')).alias('ind'))
max_grouped = grouped.groupBy("ind").agg(F.max("fail_date"))
f_range = grouped.groupBy("ind").agg(F.min("fail_date")).join(max_grouped, "ind").select(lit("failed").alias("period_state"), "*")


grouped_s = s.where(date_format("success_date","y")==2019).select("*", date_format("success_date", "D").alias('day_of_year'), row_number().over(window_spec_s).alias("rn")).select("success_date", (col('day_of_year')-col('rn')).alias('ind'))
max_grouped_s = grouped_s.groupBy("ind").agg(F.max("success_date"))
grouped_s.groupBy("ind").agg(F.min("success_date")).join(max_grouped_s, "ind").withColumnRenamed("min(success_date)","start_time").withColumnRenamed("max(success_date)","end_time").select(lit("succeeded").alias("period_state"), "*").unionAll(f_range).select("period_state", "start_time", 'end_time').show()


22/07/11 14:08:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/11 14:08:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/11 14:08:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/11 14:08:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------+----------+----------+
|period_state|start_time|  end_time|
+------------+----------+----------+
|   succeeded|2019-01-01|2019-01-03|
|   succeeded|2019-01-06|2019-01-06|
|      failed|2019-01-04|2019-01-05|
+------------+----------+----------+

